<a href="https://colab.research.google.com/github/Zehando/business-kpi-dashboards/blob/main/Sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Loading data

In [ ]:
import pandas as pd

df = pd.read_csv("/GoExplore - dailysalestable.csv")
display(df.head())

##cleaning date column

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
monthly_sales_df = df.groupby([df['Date'].dt.year, df['Date'].dt.month])['Total sale price'].sum()
monthly_sales_df.index.names = ['Year', 'Month']
monthly_sales_df = monthly_sales_df.reset_index()
monthly_sales_df.columns = ['Year', 'Month', 'Total_Sales']
monthly_sales_df = monthly_sales_df.sort_values(by=['Year', 'Month']).reset_index(drop=True)
display(monthly_sales_df.head())
display(monthly_sales_df.tail())

##Plotting data, training model and forecasting sales for rest of the year.

here I have used a Seasonal Exponential Smoothing (Holt-Winters) model was trained on the historical data to capture the trend and seasonality.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(monthly_sales_df['Year'].astype(str) + '-' + monthly_sales_df['Month'].astype(str), monthly_sales_df['Total_Sales'])
plt.title('Monthly Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model = ExponentialSmoothing(monthly_sales_df['Total_Sales'], seasonal='add', seasonal_periods=12)
fitted_model = model.fit()

In [ ]:
forecast_2018_rest = fitted_model.forecast(steps=5)
print("Forecasted monthly sales for the rest of 2018:")
display(forecast_2018_rest.apply(lambda x: f'${x:,.2f}'))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, ax = plt.subplots(figsize=(12, 6))

# Plot historical sales
historical_dates = monthly_sales_df['Year'].astype(str) + '-' + monthly_sales_df['Month'].astype(str).str.zfill(2)
ax.plot(historical_dates, monthly_sales_df['Total_Sales'], label='Historical Sales')

# Plot forecasted sales
last_historical_date_str = historical_dates.iloc[-1]
last_historical_date = pd.to_datetime(last_historical_date_str)
forecast_dates = pd.date_range(start=last_historical_date + pd.DateOffset(months=1), periods=len(forecast_2018_rest), freq='MS')
forecast_dates_str = forecast_dates.strftime('%Y-%m')
ax.plot(forecast_dates_str, forecast_2018_rest, label='Forecasted Sales', linestyle='--')

ax.set_title('Historical and Forecasted Monthly Sales')
ax.set_xlabel('Date')
ax.set_ylabel('Total Sales')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()